## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Gizo,SB,-8.10,156.84,80.98,77,90,7.00,moderate rain
1,1,Louis Trichardt,ZA,-23.04,29.90,66.70,75,96,1.95,light rain
2,2,Lufilufi,WS,-13.87,-171.60,77.00,94,40,4.70,scattered clouds
3,3,Cape Town,ZA,-33.93,18.42,64.00,87,75,19.46,broken clouds
4,4,Beyneu,KZ,45.32,55.20,45.21,54,0,11.50,clear sky


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Gizo,SB,-8.10,156.84,80.98,77,90,7.00,moderate rain
2,2,Lufilufi,WS,-13.87,-171.60,77.00,94,40,4.70,scattered clouds
6,6,Bernalillo,US,35.05,-106.67,78.01,14,1,6.93,clear sky
9,9,Kinkala,CG,-4.36,14.76,80.60,57,40,10.29,thunderstorm
13,13,Dingle,PH,11.00,122.67,76.69,90,100,2.82,overcast clouds
18,18,Kulhudhuffushi,MV,6.62,73.07,81.81,76,11,11.43,few clouds
24,24,Port Said,EG,31.26,32.28,80.60,74,0,14.99,clear sky
25,25,Salalah,OM,17.02,54.09,78.80,78,31,2.24,scattered clouds
27,27,Duncan,US,34.50,-97.96,77.00,56,1,4.70,clear sky
34,34,Mitsamiouli,KM,-11.38,43.28,78.31,74,65,2.75,broken clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                211
City                   211
Country                210
Lat                    211
Lng                    211
Max Temp               211
Humidity               211
Cloudiness             211
Wind Speed             211
Current Description    211
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                210
City                   210
Country                210
Lat                    210
Lng                    210
Max Temp               210
Humidity               210
Cloudiness             210
Wind Speed             210
Current Description    210
dtype: int64

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Gizo,SB,80.98,moderate rain,-8.10,156.84,
2,Lufilufi,WS,77.00,scattered clouds,-13.87,-171.60,
6,Bernalillo,US,78.01,clear sky,35.05,-106.67,
9,Kinkala,CG,80.60,thunderstorm,-4.36,14.76,
13,Dingle,PH,76.69,overcast clouds,11.00,122.67,
18,Kulhudhuffushi,MV,81.81,few clouds,6.62,73.07,
24,Port Said,EG,80.60,clear sky,31.26,32.28,
25,Salalah,OM,78.80,scattered clouds,17.02,54.09,
27,Duncan,US,77.00,clear sky,34.50,-97.96,
34,Mitsamiouli,KM,78.31,broken clouds,-11.38,43.28,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
      
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
hotel_df.count()

City                   210
Country                210
Max Temp               210
Current Description    210
Lat                    210
Lng                    210
Hotel Name             210
dtype: int64

In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   210
Country                210
Max Temp               210
Current Description    210
Lat                    210
Lng                    210
Hotel Name             210
dtype: int64

In [16]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Gizo,SB,80.98,moderate rain,-8.10,156.84,Imagination Island
2,Lufilufi,WS,77.00,scattered clouds,-13.87,-171.60,Leuaina Beach Resort and Spa
6,Bernalillo,US,78.01,clear sky,35.05,-106.67,DoubleTree by Hilton Hotel Albuquerque
9,Kinkala,CG,80.60,thunderstorm,-4.36,14.76,Hotel la Vernande
13,Dingle,PH,76.69,overcast clouds,11.00,122.67,Camp Pasica
...,...,...,...,...,...,...,...
702,San Andres,CO,78.80,light intensity drizzle,12.58,-81.70,Hotel Arena Blanca
705,Mahenge,TZ,75.07,scattered clouds,-8.68,36.72,Mkondosii Guest House
706,Zaraza,VE,84.49,moderate rain,9.35,-65.32,Hotel Zaraza
707,Corona,US,86.00,haze,33.88,-117.57,Holiday Inn Express & Suites Corona


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 

locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))